In [2]:

from belly import *


In [513]:
import pickle
from gcsfs import GCSFileSystem

new_schema = create_schema()
fs = GCSFileSystem(project='trollhunters')

with fs.open('gs://spain-tweets/schemas/tweet-clean.pickle', 'wb') as f:
    pickle.dump(new_schema, f)